In [1]:
import pandas as pd
import os
import numpy as np
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

data_path = 'data'

In [6]:
# import HSI Index minute data and HKDCNY minute data
index = 'HSTECH'
etf = '513180'

df_index = pl.read_csv(os.path.join(data_path, f'{index}.csv'))
HKDCNH = pl.read_csv(os.path.join(data_path, 'HKDCNH.csv')) # use HKDCNH

df_index = df_index.with_columns(
    pl.col("Datetime").str.strptime(pl.Datetime).dt.convert_time_zone("Asia/Hong_Kong").dt.replace_time_zone(None)
)

HKDCNH = HKDCNH.with_columns(
    pl.col("Datetime").str.strptime(pl.Datetime).dt.convert_time_zone("Asia/Hong_Kong").dt.replace_time_zone(None)
)

df_index= df_index.sort('Datetime')

df_index = df_index.with_columns(
    pl.col('Datetime').cast(pl.Date).alias('Date')
)

df_index = df_index.join(HKDCNH.select(pl.col('Close').alias('HKDCNY'), pl.col('Datetime')), on='Datetime', how='left')

df_index = df_index.with_columns(
    pl.col('HKDCNY').fill_null(1).backward_fill()
)

# convert HSI index to CNY
df_index = df_index.with_columns(
    (pl.col('Close')*pl.col('HKDCNY')).alias('CloseCNY'),
)


# choose only datetime between 9:40 to 16:00
df_index = df_index.filter(
    pl.col('Datetime').dt.time().is_between(
        pl.time(9, 30),
        pl.time(16, 00)
    )
)

# calculate HSI index 1m return
df_index = df_index.with_columns(
    (pl.col('CloseCNY')/pl.col('CloseCNY').shift(1) - 1).alias('Return').fill_null(0),
)

df_index.group_by('Date').agg(
    pl.col('Datetime').first()
)




# import NAV data, this is daily end of day NAV data, we need to use index minute data to estimate 1m NAV of every next day
nav= pl.read_excel(os.path.join(data_path, f'nav_{etf}.xlsx'))

# we set the NAV of the previous day to be the NAV of next day start at 9:40
nav = nav.sort('Date').with_columns(
    pl.col('net_nav').shift(1).alias('pre_unit_nav'),
)
nav = nav.with_columns(
    (pl.col('Date').cast(pl.Datetime) + pl.duration(hours=9, minutes=40)).alias('Datetime'),
)

# now, the 1m data has every start NAV of every day, which is the EOD NAV of previous day
df_index = df_index.join(nav.select(pl.col('Datetime'), pl.col('pre_unit_nav')), on='Datetime', how='left').rename({'pre_unit_nav': 'est_iopv'})

# calculate the estimated NAV of every minute by useing start NAV of every day * (1+1m return of index)
df_index = df_index.with_columns(
    pl.col('est_iopv').forward_fill(),
    (pl.col('Return')+1).cum_prod().over('Date').alias('day_refresh_cum_return'),
).with_columns(
    pl.col('est_iopv')*pl.col('day_refresh_cum_return').alias('est_iopv'),
)

# df_index.group_by('Date').agg(
#     pl.col('Datetime').last()
# )

# remove first date because without previous 
df_index = df_index.filter(
    pl.col('Date') > pl.col('Date').first()
)

df = pl.read_csv(os.path.join(data_path, f'{etf}.csv'))
df = df.with_columns(
    pl.col('datetime').str.strptime(pl.Datetime).alias('Datetime')
).join(df_index.select(['Datetime', 'est_iopv','CloseCNY']).rename({'CloseCNY':'index_close'}), on='Datetime', how='inner')


df = (
    df
    .with_columns(
    # premium rate
    ((pl.col('close')-pl.col('est_iopv'))/pl.col('est_iopv')).alias('PD')
    )
    .with_columns(
        #rolling 5 min PD
        pl.col('PD').rolling_mean(10).alias('ma_PD'),
    )
    .with_columns(
        #studied PD jump
        (pl.col('PD') - pl.col('ma_PD')).alias('jump_PD'),
    )
)

# create 1- 20 min future return
df = df.with_columns(
    #[(pl.col('close').shift(-i)/pl.col('close')).alias(f'future_return_{i}') for i in range(1, 21)],
    [((pl.col('close').shift(-i)/pl.col('close'))/(pl.col('index_close').shift(-i)/pl.col('index_close'))).alias(f'future_return_{i}') for i in range(1, 21)]
)

df



datetime,close,total_turnover,low,iopv,open,num_trades,high,volume,Datetime,est_iopv,index_close,PD,ma_PD,jump_PD,future_return_1,future_return_2,future_return_3,future_return_4,future_return_5,future_return_6,future_return_7,future_return_8,future_return_9,future_return_10,future_return_11,future_return_12,future_return_13,future_return_14,future_return_15,future_return_16,future_return_17,future_return_18,future_return_19,future_return_20
str,f64,f64,f64,f64,f64,f64,f64,f64,datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""2025-04-07 09:31:00""",0.679,4.51627938e8,0.668,0.6703,0.675,13721.0,0.68,6.703725e8,2025-04-07 09:31:00,0.621676,4469.088955,0.092209,null,null,0.984629,0.975883,0.979185,0.988845,0.977851,0.985363,0.98756,0.983322,0.98013,0.985686,0.979822,0.97534,0.98332,0.989136,0.985168,0.989021,0.982834,0.985542,0.986509,0.987997
"""2025-04-07 09:32:00""",0.673,1.29937191e8,0.672,0.6601,0.674,4992.0,0.676,1.92885838e8,2025-04-07 09:32:00,0.625802,4498.749453,0.07542,null,null,0.991118,0.994472,1.004282,0.993117,1.000746,1.002977,0.998673,0.995431,1.001074,0.995118,0.990566,0.998671,1.004578,1.000547,1.004461,0.998177,1.000928,1.00191,1.003421,0.998024
"""2025-04-07 09:33:00""",0.67,1.3246021e8,0.67,0.6641,0.673,3485.0,0.673,1.973675e8,2025-04-07 09:33:00,0.628595,4518.831491,0.065868,null,null,1.003384,1.013282,1.002016,1.009714,1.011966,1.007623,1.004352,1.010045,1.004036,0.999443,1.00762,1.013581,1.009514,1.013462,1.007123,1.009898,1.010889,1.012413,1.006968,1.001929
"""2025-04-07 09:34:00""",0.672,1.01529206e8,0.669,0.6661,0.669,2468.0,0.675,1.51001e8,2025-04-07 09:34:00,0.628346,4517.035922,0.069475,null,null,1.009865,0.998637,1.006309,1.008553,1.004224,1.000965,1.006639,1.00065,0.996073,1.004222,1.010162,1.006109,1.010044,1.003726,1.006492,1.00748,1.008999,1.003572,0.99855,0.998395
"""2025-04-07 09:35:00""",0.673,1.06923713e8,0.672,0.6651,0.672,2059.0,0.674,1.590132e8,2025-04-07 09:35:00,0.623133,4479.567355,0.080025,null,null,0.988882,0.996479,0.998701,0.994415,0.991187,0.996806,0.990875,0.986343,0.994412,1.000295,0.996281,1.000178,0.993921,0.99666,0.997638,0.999143,0.993768,0.988795,0.988642,0.992742
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2025-04-11 14:56:00""",0.681,1.6401282e7,0.681,0.6852,0.681,396.0,0.682,2.40664e7,2025-04-11 14:56:00,0.674023,4638.631934,0.010351,0.012242,-0.001891,1.001652,0.999845,1.001539,1.00069,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2025-04-11 14:57:00""",0.682,3.5110301e7,0.681,0.6855,0.682,761.0,0.682,5.15175e7,2025-04-11 14:57:00,0.6739,4637.783937,0.01202,0.0123,-0.000281,0.998197,0.999888,0.99904,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2025-04-11 14:58:00""",0.682,3.3856862e7,0.681,0.6857,0.682,794.0,0.682,4.96952e7,2025-04-11 14:58:00,0.675117,4646.161381,0.010195,0.012091,-0.001896,1.001694,1.000844,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [7]:
# 1d iopv moves 
# index minute close and ETF minute close last 1s? 
# bbg index/futures basus  (HSI Index HIJ5 Index SGIP10)


In [8]:
df = df.with_columns(
    (pl.col('close')/pl.col('close').shift(1)).cum_prod().alias('etf return'),
    (pl.col('index_close')/pl.col('index_close').shift(1)).cum_prod().alias('index return')
)

In [10]:
fig = go.Figure()
df_filtered = df
          
# Add ETF return line
fig.add_trace(go.Scatter(
   x=df_filtered['Datetime'].dt.strftime("%Y-%m-%d %H:%M:%S").to_list(),
   y=df_filtered['etf return'],
   mode='lines',
   name=f'ETF',
   line=dict(color='blue')
))

# Add Index return line
fig.add_trace(go.Scatter(
   x=df_filtered['Datetime'].dt.strftime("%Y-%m-%d %H:%M:%S").to_list(),
   y=df_filtered['index return'],
   mode='lines',
   name=f'Index',
   line=dict(color='green')
))

# Add ETF - Index return line on secondary y-axis
fig.add_trace(go.Scatter(
   x=df_filtered['Datetime'].dt.strftime("%Y-%m-%d %H:%M:%S").to_list(),
   y=(df_filtered['etf return'] - df_filtered['index return']),
   mode='lines',
   name=f'ETF - Index',
   line=dict(color='red'),
   yaxis='y2'
))

# Update layout for secondary y-axis
fig.update_layout(
   title=f'{etf} vs {index}',
   xaxis=dict(title='Datetime', type='category'),
   yaxis=dict(title='Return'),
   yaxis2=dict(title='ETF - Index', overlaying='y', side='right'),
   legend=dict(title='Legend'),
   height=800,
   width=1200,
)

fig


In [320]:

# plot distribution of jump_PD using plotly
fig = px.histogram(df.to_pandas(), x='jump_PD', nbins=100)
fig.update_layout(
    title='Distribution of PD Jump',
    xaxis_title='PD Jump',
    yaxis_title='Count',
    bargap=0.2
)
fig.show()


In [321]:
fig = go.Figure()

for bound, color in zip([0.0005, 0.001, 0.0015], ['blue', 'green', 'red']):
    # Calculate mean future returns for positive PD jumps
    result = df.filter(pl.col('jump_PD') <= -bound).select(
        [(pl.col(f'future_return_{i}') - 1).mean() for i in range(1, 21)]
    ).transpose()
    # Convert to Pandas DataFrame for plotting
    result_pd = result.to_pandas()

    # Add line to the figure
    fig.add_trace(go.Scatter(
        x=[i for i in range(1, 21)],
        y=result_pd['column_0'],
        mode='lines',
        name=f'<= -{bound}',
        line=dict(color=color)
    ))


# Update layout
fig.update_layout(
    title='Mean Future Returns for Positive PD Jumps',
    xaxis_title='Future x minute return',
    yaxis_title='Mean Value',
    legend_title='Bounds'
)

fig.show()


In [322]:
fig = go.Figure()

for bound, color in zip([0.0005, 0.001, 0.0015], ['blue', 'green', 'red']):
    # Calculate mean future returns for positive PD jumps
    result = df.filter(pl.col('jump_PD') >= bound).select(
        [(pl.col(f'future_return_{i}') - 1).mean() for i in range(1, 21)]
    ).transpose()
    # Convert to Pandas DataFrame for plotting
    result_pd = result.to_pandas()

    # Add line to the figure
    fig.add_trace(go.Scatter(
        x=[i for i in range(1, 21)],
        y=result_pd['column_0'],
        mode='lines',
        name=f'>= {bound}',
        line=dict(color=color)
    ))

# Update layout
fig.update_layout(
    title='Mean Future Returns for Positive PD Jumps',
    xaxis_title='Future x minute return',
    yaxis_title='Mean Value',
    legend_title='Bounds'
)

fig.show()
